In [ ]:
import praw
import json
import time
import random
import os

#Giving the credentials using PRAW
reddit = praw.Reddit(
    client_id='94-KDboZSbfIo3SK3FDSzg',         
    client_secret='6ktVmT5--Uj7CeubVKIJJCII2tFsEQ',
    user_agent='DataScrapper1811'
)

# Configuration
subreddits = ['python', 'datascience', 'AskReddit']
POST_LIMIT_PER_SUB = 500
CHUNK_SIZE = 100
INCLUDE_COMMENTS = True

all_data = []
file_index = 1

# Creating an output folder
output_dir = "reddit_scrape_chunks"
os.makedirs(output_dir, exist_ok=True)

# Beginning scraping
for sub in subreddits:
    print(f" Scraping r/{sub}...")

    try:
        for count, post in enumerate(reddit.subreddit(sub).top(limit=POST_LIMIT_PER_SUB), 1):
            try:
                post_data = {
                    'subreddit': sub,
                    'title': post.title,
                    'id': post.id,
                    'url': post.url,
                    'score': post.score,
                    'text': post.selftext,
                    'created': post.created_utc,
                    'num_comments': post.num_comments,
                    'comments': []
                }

                if INCLUDE_COMMENTS:
                    post.comments.replace_more(limit=0)
                    post_data['comments'] = [{
                        'id': comment.id,
                        'author': str(comment.author),
                        'body': comment.body,
                        'score': comment.score,
                        'created': comment.created_utc
                    } for comment in post.comments.list()]

                all_data.append(post_data)

                # Saving in chunks
                if count % CHUNK_SIZE == 0:
                    file_path = os.path.join(output_dir, f"{sub}_batch_{file_index}.json")
                    with open(file_path, "w", encoding="utf-8") as f:
                        json.dump(all_data, f, indent=2)
                    print(f" Saved {len(all_data)} posts to {file_path}")
                    all_data = []
                    file_index += 1

                time.sleep(0.5)  # slight delay per post

            except Exception as post_err:
                print(f" Error on post {post.id}: {post_err}")
                time.sleep(2)

        # Pausing between subreddits
        sleep_time = random.uniform(6, 12)
        print(f" Sleeping {sleep_time:.2f}s to avoid rate limit...")
        time.sleep(sleep_time)

    except Exception as e:
        print(f" Error in r/{sub}: {e}")
        if "429" in str(e):
            print(" Hit rate limit. Sleeping for 60s...")
            time.sleep(60)
        else:
            time.sleep(5)

# Final saving if anything is left
if all_data:
    file_path = os.path.join(output_dir, f"{sub}_batch_{file_index}.json")
    with open(file_path, "w", encoding="utf-8") as f:
        json.dump(all_data, f, indent=2)
    print(f" Final save: {len(all_data)} posts to {file_path}")
